## Task 5

To begin, a folder was added to the src folder named sojourn. Within this folder, two files were added: tandem_netowrk and sojourn_engine. Within tandem_network, the framework built for the original simulation network was implemented and adjusted for this new case. The fields for QueueNetworkState (Line 54) were changed to keep track of individual jobs in the system.

In [ ]:
mutable struct QueueNetworkState <: State 
    jobs_num::Vector{Vector{Int}} # Jobs in each queue. 
    params::NetworkParameters # Parameters of queue network system
end 

We will then treat the jobs as a vector of vectors. Each inner vector being the queue where jobs are being served, and at these queues are the individual jobs, labeled by integers. We intend to label jobs with integers as they enter the system. To assist with this, we can introduce two global variables to keep track of the total job arrivals, as well as the total jobs currently in the system.

In [ ]:
global new_job = 0
global total_jobs = 0 
global Dict{Int, boolean}()

This last variable will be a dictionary of all the jobs in the system. They value (boolean) associated with a key (job) will be true only if the job is in the system. 

Processing an event for an external arrival was re-written for this scenario (Line 131).

In [ ]:
"""
Process external arrival events that occur initially in the system.
"""
function process_event(time::Float64, state::State, ext_event::ExternalArrivalEvent) 
    q = ext_event.next_q # Queue where job is added 

    new_job += 1 #there is a new job in the system
    total_jobs += 1
    dict_jobs[total_jobs] = true #add job to jobs dictionary for system
    push!(state.jobs_num[q], new_job) # Add job to queue 
    
    new_timed_events = TimedEvent[] # Record a new timed event 
    # Prepare for next arrival
    push!(new_timed_events, TimedEvent(ExternalArrivalEvent(q), 
                                        time + next_arrival_duration(state, q)))
    # Start new service event, since this will always be first job in queue 
    # If this job is only job in queue then start new service event 
    lenght(state.jobs_num[q]) == 1 && push!(new_timed_events, TimedEvent(EndOfServiceAtQueueEvent(q, nothing), 
                                                    time + next_service_duration(state, q)))
    return new_timed_events
end 